# Activity Prediction with P-adic Embeddings

This notebook demonstrates how to use the Ternary VAE's p-adic embeddings
to predict antimicrobial peptide activity.

## Contents
1. Load the Codon Encoder
2. Embed Peptide Sequences
3. Train an Activity Predictor
4. Predict Activity for New Sequences
5. Analyze Embedding Space

In [ ]:
# Setup paths
import sys
from pathlib import Path

# Add project paths
deliverables_path = Path.cwd().parent
project_root = deliverables_path.parent
sys.path.insert(0, str(deliverables_path))
sys.path.insert(0, str(project_root))

import numpy as np
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Load the Codon Encoder

The codon encoder maps amino acid sequences into a 16-dimensional hyperbolic
embedding space where radial position encodes 3-adic valuation.

In [ ]:
from shared import CodonEncoder

# Initialize the codon encoder
# This loads the pretrained VAE checkpoint
encoder = CodonEncoder()
print(f"Codon encoder initialized")
print(f"  Latent dimension: {encoder.latent_dim}")
print(f"  Device: {encoder.device}")

## 2. Embed Peptide Sequences

Convert peptide sequences into p-adic embeddings. Each amino acid is mapped
to a point in hyperbolic space, and the sequence embedding is computed.

In [ ]:
# Example antimicrobial peptides with known activity
peptides = {
    "Magainin 2": {"sequence": "GIGKFLHSAKKFGKAFVGEIMNS", "mic": 10.0},
    "Melittin": {"sequence": "GIGAVLKVLTTGLPALISWIKRKRQQ", "mic": 1.0},
    "LL-37": {"sequence": "LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES", "mic": 5.0},
    "Indolicidin": {"sequence": "ILPWKWPWWPWRR", "mic": 8.0},
    "Cecropin A": {"sequence": "KWKLFKKIEKVGQNIRDGIIKAGPAVAVVGQATQIAK", "mic": 2.0},
    "Defensin": {"sequence": "ACYCRIPACIAGERRYGTCIYQGRLWAFCC", "mic": 15.0},
    "Cathelicidin": {"sequence": "GRFKRFRKKFKKLFKKLSPVIPLLHL", "mic": 4.0},
    "Thanatin": {"sequence": "GSKKPVPIIYCNRRTGKCQRM", "mic": 12.0},
}

# Compute embeddings
embeddings = {}
for name, data in peptides.items():
    emb = encoder.encode_sequence(data["sequence"])
    embeddings[name] = {
        "embedding": emb,
        "radius": np.linalg.norm(emb),
        "mic": data["mic"],
    }
    print(f"{name:<15} radius={embeddings[name]['radius']:.4f}, MIC={data['mic']:.1f} uM")

In [ ]:
# Visualize the embedding space
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Extract data for plotting
names = list(embeddings.keys())
radii = [embeddings[n]["radius"] for n in names]
mics = [embeddings[n]["mic"] for n in names]

# Compute correlation
corr, pval = spearmanr(radii, mics)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(radii, mics, s=100, c=np.log10(mics), cmap="viridis")

for i, name in enumerate(names):
    ax.annotate(name, (radii[i], mics[i]), fontsize=9, ha="left", va="bottom")

ax.set_xlabel("Embedding Radius", fontsize=12)
ax.set_ylabel("MIC (uM)", fontsize=12)
ax.set_title(f"P-adic Radius vs Antimicrobial Activity\nSpearman rho={corr:.2f}, p={pval:.3f}", fontsize=14)
plt.colorbar(scatter, label="log10(MIC)")
plt.tight_layout()
plt.show()

## 3. Train an Activity Predictor

Use the p-adic embeddings as features for predicting antimicrobial activity.
We'll use a simple random forest regressor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error, r2_score

# Prepare training data
X = np.array([embeddings[n]["embedding"] for n in names])
y = np.log10(np.array([embeddings[n]["mic"] for n in names]))  # Log-transform MIC

print(f"Training data shape: X={X.shape}, y={y.shape}")

# Train model with leave-one-out cross-validation
loo = LeaveOneOut()
model = RandomForestRegressor(n_estimators=100, random_state=42)

y_pred = np.zeros_like(y)
for train_idx, test_idx in loo.split(X):
    model.fit(X[train_idx], y[train_idx])
    y_pred[test_idx] = model.predict(X[test_idx])

# Evaluate
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)
print(f"\nLeave-one-out cross-validation:")
print(f"  RMSE: {rmse:.3f} log10(uM)")
print(f"  R2: {r2:.3f}")

In [ ]:
# Plot predictions vs actual
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(y, y_pred, s=100, alpha=0.7)

# Add peptide names
for i, name in enumerate(names):
    ax.annotate(name, (y[i], y_pred[i]), fontsize=9, ha="left", va="bottom")

# Add diagonal line
lims = [min(y.min(), y_pred.min()) - 0.1, max(y.max(), y_pred.max()) + 0.1]
ax.plot(lims, lims, "k--", alpha=0.5, label="Perfect prediction")

ax.set_xlabel("Actual log10(MIC)", fontsize=12)
ax.set_ylabel("Predicted log10(MIC)", fontsize=12)
ax.set_title(f"Activity Prediction (LOO CV)\nRMSE={rmse:.3f}, R2={r2:.3f}", fontsize=14)
ax.legend()
ax.set_xlim(lims)
ax.set_ylim(lims)
plt.tight_layout()
plt.show()

## 4. Predict Activity for New Sequences

Use the trained model to predict activity for novel peptide sequences.

In [ ]:
# Train final model on all data
final_model = RandomForestRegressor(n_estimators=100, random_state=42)
final_model.fit(X, y)

# Predict for new sequences
new_peptides = [
    "KLWKKWKKWLK",      # Synthetic cationic peptide
    "GIGKFLHSAGK",      # Magainin variant
    "RRWWRRWWRR",       # Arg-Trp rich peptide
]

print("Predictions for new peptides:")
print("-" * 50)
for seq in new_peptides:
    emb = encoder.encode_sequence(seq)
    log_mic_pred = final_model.predict(emb.reshape(1, -1))[0]
    mic_pred = 10 ** log_mic_pred
    radius = np.linalg.norm(emb)
    print(f"{seq:<20} radius={radius:.4f}, predicted MIC={mic_pred:.1f} uM")

## 5. Analyze Embedding Space

Visualize the p-adic embedding dimensions to understand what features
the VAE has learned.

In [ ]:
# Analyze feature importance
importances = final_model.feature_importances_

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(range(len(importances)), importances)
ax.set_xlabel("Embedding Dimension", fontsize=12)
ax.set_ylabel("Feature Importance", fontsize=12)
ax.set_title("P-adic Embedding Dimension Importance for Activity Prediction", fontsize=14)
ax.set_xticks(range(len(importances)))
plt.tight_layout()
plt.show()

# Top dimensions
top_dims = np.argsort(importances)[::-1][:5]
print(f"\nTop 5 important dimensions: {top_dims}")
for dim in top_dims:
    print(f"  Dim {dim}: importance={importances[dim]:.3f}")

In [ ]:
# Reduce to 2D for visualization using PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_2d = pca.fit_transform(X)

fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(X_2d[:, 0], X_2d[:, 1], c=y, s=200, cmap="coolwarm", edgecolor="k")

for i, name in enumerate(names):
    ax.annotate(name, (X_2d[i, 0], X_2d[i, 1]), fontsize=10, ha="left", va="bottom")

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)", fontsize=12)
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)", fontsize=12)
ax.set_title("Peptide Embedding Space (PCA)", fontsize=14)
plt.colorbar(scatter, label="log10(MIC)")
plt.tight_layout()
plt.show()

## 6. Combine with Biophysical Features

Combine p-adic embeddings with traditional biophysical features for
improved predictions.

In [ ]:
from shared import compute_peptide_properties, compute_ml_features

# Combine features
def get_combined_features(sequence, encoder):
    """Get combined p-adic + biophysical features."""
    # P-adic embedding
    padic_emb = encoder.encode_sequence(sequence)
    
    # Biophysical features
    props = compute_peptide_properties(sequence)
    bio_features = np.array([
        props["length"],
        props["net_charge"],
        props["hydrophobicity"],
        props["hydrophobic_ratio"],
        props["cationic_ratio"],
    ])
    
    return np.concatenate([padic_emb, bio_features])

# Build combined feature matrix
X_combined = np.array([get_combined_features(peptides[n]["sequence"], encoder) for n in names])
print(f"Combined feature shape: {X_combined.shape}")
print(f"  P-adic dimensions: {X.shape[1]}")
print(f"  Biophysical features: 5")

In [ ]:
# Compare models
from sklearn.ensemble import GradientBoostingRegressor

models = {
    "P-adic only": (X, RandomForestRegressor(n_estimators=100, random_state=42)),
    "Combined": (X_combined, RandomForestRegressor(n_estimators=100, random_state=42)),
    "Combined (GB)": (X_combined, GradientBoostingRegressor(n_estimators=100, random_state=42)),
}

print("Model comparison (LOO CV):")
print("-" * 50)

for name, (features, model) in models.items():
    y_pred = np.zeros_like(y)
    for train_idx, test_idx in loo.split(features):
        model.fit(features[train_idx], y[train_idx])
        y_pred[test_idx] = model.predict(features[test_idx])
    
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    print(f"{name:<20} RMSE={rmse:.3f}, R2={r2:.3f}")

## Summary

This notebook demonstrated:

1. **Codon Encoder**: Loading and using the pretrained p-adic encoder
2. **Sequence Embedding**: Converting peptides to 16-dimensional vectors
3. **Activity Prediction**: Training ML models on embeddings
4. **Novel Predictions**: Predicting activity for new sequences
5. **Feature Analysis**: Understanding which dimensions are important
6. **Feature Combination**: Combining p-adic with biophysical features

### Key Findings

- The p-adic embedding radius correlates with antimicrobial activity
- Combining p-adic embeddings with biophysical features improves predictions
- Certain embedding dimensions capture activity-relevant information

### Next Steps

- Train on larger datasets (DRAMP, APD3)
- Add uncertainty quantification
- Use for peptide optimization with genetic algorithms